In [29]:
library(NELSI)
library(phangorn)

In [30]:
ml_dat <- read.table('nonstatlong.1.Lboot.gtrg.cvs', head = T, sep = ',', stringsAsFactors = F)

In [89]:
c.rec <- function(b_list){
    if(length(b_list) == 1){
        return(b_list)
    }else if(length(b_list) == 2){
        return(c(b_list[[1]], b_list[[2]]))
    }else{
        return(c(b_list[[1]], c.rec(b_list[-1])))
    }
}

In [90]:
trees <- lapply(ml_dat$tree, function(tr) read.tree(text = tr))

In [91]:
tl <- sapply(trees, function(tr) sum(tr$edge.length))

In [96]:
boot_files <- dir(pattern = 'cvs')
for(b in boot_files){
    folder_name <- gsub('[.]cvs', '_folder', b)
    system(paste('mkdir', folder_name))
    boot_data <- read.table(b, head = T, sep = ',', stringsAsFactors = F)
    
    if(ncol(boot_data) == 1){
        for(n in 1:nrow(boot_data)){
            tr <- read.tree(text = boot_data$tree[n])
            seq <- as.DNAbin(simSeq(tr))
            write.dna(seq, paste0(folder_name, '/', 'rep', n, '.phy'))
        }
    }else{
        for(n in 1:nrow(boot_data)){
            tr <- read.tree(text = boot_data$tree[n])
            rates <- phangorn::discrete.gamma(alpha = boot_data$alpha[n], k = 4) 
            if(any(rates < 0.01)) rates <- rates + 0.01
            seq_list <- lapply(rates, function(x) simSeq(tr, 
                    Q = as.numeric(boot_data[n, c('ac', 'ag', 'at', 'cg', 'ct', 'gt')]),
                    bf = as.numeric(boot_data[n, c('a', 'c', 'g', 't')]), l = 250, rate = x))
            seq <- as.DNAbin(c.rec(seq_list))
            write.dna(seq, paste0(folder_name, '/', 'rep', n, '.phy'))
            }
    }
}

In [93]:
seq

19 sequences with 1000 character and 196 different site patterns.
The states are a c g t 